# AI in Web Programming — Labs Outline (Fall 2025)

*This notebook provides a week‑by‑week outline, acceptance criteria, and command snippets for a semester focused on FastAPI + React (CRA), TensorFlow.js (in‑browser), SQLite + FAISS vector search, and Gemini APIs via a backend proxy. It is written intentionally in passive voice and without STT/TTS.*

**Stack**
- Backend: FastAPI (Python 3.11), `uvicorn`, `pydantic`, `faiss-cpu`, `python-dotenv`, `google-generativeai`
- Frontend: React (Create React App)
- Data: SQLite (+ FAISS index files on disk)
- LLMs/Embeddings: Gemini API (`gemini-1.5-flash`, `text-embedding-004`)
- Vector store: FAISS (or `sqlite-vss` alternative if chosen later)

**Repository layout**
```
ai-web/
  backend/
  frontend/
  data/
  reports/
  .env.example
  README.md
```


## Setup — One-time

Back end environment creation:

```bash
cd ai-web/backend
python -m venv .venv
. .venv/bin/activate  # Windows: .venv\Scripts\activate
pip install fastapi uvicorn[standard] pydantic python-dotenv google-generativeai faiss-cpu numpy
pip freeze > requirements.txt
```

Front end creation:

```bash
cd ai-web
npx create-react-app frontend
cd frontend
npm install
```

Environment template (`.env.example` at repo root) should be created with:

```
GEMINI_API_KEY=YOUR_KEY_HERE
API_BASE_URL=http://localhost:8000
REACT_APP_API_BASE=http://localhost:8000
```

When running locally, `.env` files should be placed under `backend/` and `frontend/` as needed.


## Week 1 — Git, GitHub, React + FastAPI “Hello”

**Goals**
- A CRA app and a FastAPI server are stood up; commits are pushed to GitHub.

**Tasks**
- `GET /health` and `POST /echo` endpoints are implemented.
- CORS is enabled for development.

**Checks**
- `curl http://localhost:8000/health` returns `{"status":"ok"}`.
- The React page calls `/echo` and renders the uppercase message.


## Week 2 — HTTP & Forms

**Goals**
- Robust fetch flows are created with loading, error, and retry states.

**Tasks**
- A controlled React form is implemented.
- A reusable `frontend/src/lib/api.js` helper is added.

**Checks**
- A form submission shows server response and handles failures cleanly.


## Week 3 — Gemini chat via FastAPI proxy

**Goals**
- `/api/chat` in FastAPI proxies a Gemini request; no keys exist in the browser.

**Tasks**
- `google.generativeai` is configured with `GEMINI_API_KEY` from env.
- A simple chat box in React calls `/api/chat` and renders the reply.

**Checks**
- A conversation round trip functions with the proxy only.


## Week 4 — Prompting to JSON + Validation

**Goals**
- Structured outputs are requested and validated with Pydantic.

**Tasks**
- A planner prompt returns `{goal: str, steps: list[str]}` for a user intent.
- FastAPI validates and repairs malformed JSON with a follow-up prompt.

**Checks**
- Invalid model output does not crash; a valid JSON object is always returned.


## Week 5 — Tools & Simple Agent Loop

**Goals**
- A minimal function-calling style agent is implemented with a short loop.

**Tools**
- `calculator`, `db.query(sql)`, and `search_faq(query)` (stub).

**Checks**
- Tool invocations and observations are rendered in a timeline UI.


## Week 6 — Embeddings & Vector Store (FAISS)

**Goals**
- Document chunks are embedded with Gemini and indexed in FAISS.

**Tasks**
- PDF/Markdown chunking is implemented.
- `text-embedding-004` produces vectors; FAISS index is written to disk.

**Checks**
- `/api/search?q=` returns top-k with cosine scores; a search UI displays results.


## Week 7 — RAG Answerer with Citations

**Goals**
- Retrieved chunks are passed to Gemini with a constrained prompt.

**Tasks**
- Citations `[S1]`, `[S2]` are inserted with snippet previews.

**Checks**
- Answers cite only provided context and refuse when evidence is missing.


## Week 8 — In-browser AI with TensorFlow.js

**Goals**
- Client-only inference is demonstrated with MobileNet or Coco-SSD.

**Tasks**
- Webcam/file upload UI is created; top-k or boxes are drawn on canvas.

**Checks**
- The page functions offline after initial load and never hits the backend.


## Week 9 — Agent Memory & Session Store (SQLite)

**Goals**
- Chat history and memory notes are persisted.

**Tasks**
- Tables: `sessions`, `messages`, `memories`.
- Periodic summarization creates a memory note that is vectorized and upserted.

**Checks**
- Conversations persist across refresh; memory viewer shows entries.


## Week 10 — Evaluation, Latency & Caching

**Goals**
- A tiny eval set is created; latency and token usage are logged.

**Tasks**
- A prompt-output cache is implemented (hash-based).
- Basic charts are rendered in the frontend reports page.

**Checks**
- A run log exists in SQLite; cache hit rate and latency medians are visible.


## Week 11 — Security & Hardening

**Goals**
- Minimum viable security is applied.

**Tasks**
- `.env` secrecy, strict CORS for production, payload size limits.
- File-type allow-listing, rate limiting with `slowapi`, and audit logs.

**Checks**
- Limit enforcement and 429 responses are verified with repeated requests.


## Week 12 — Deployment & Capstone Integration

**Goals**
- The “AI Campus Helpdesk” capstone is delivered live.

**Tasks**
- FastAPI is deployed (Render/Railway); React is built and hosted.
- `.env.example`, health checks, and README are finalized.

**Checks**
- A five-minute demo script is followed end-to-end without errors.


## Appendix — Useful snippets

**FastAPI health + CORS**
```python
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"])

@app.get("/health")
def health():
    return {"status":"ok"}
```

**Embedding texts with Gemini + FAISS**
```python
import os, numpy as np, faiss, google.generativeai as genai
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

def embed_texts(texts):
    vecs = []
    for t in texts:
        e = genai.embed_content(model="text-embedding-004", content=t)
        vecs.append(np.array(e["embedding"], dtype="float32"))
    X = np.vstack(vecs)
    faiss.normalize_L2(X)
    return X

def build_index(X):
    idx = faiss.IndexFlatIP(X.shape[1])
    idx.add(X)
    return idx
```

**React fetch helper**
```javascript
const BASE = process.env.REACT_APP_API_BASE || "http://localhost:8000";
export async function post(path, body){
  const r = await fetch(`${BASE}${path}`, {
    method: 'POST',
    headers: {'Content-Type':'application/json'},
    body: JSON.stringify(body)
  });
  if(!r.ok) throw new Error(`HTTP ${r.status}`);
  return r.json();
}
```
